In [1]:
! pip install dist/quick_sentiments-0.1.0-py3-none-any.whl

Processing c:\users\meala\documents\quick_sentiments\dist\quick_sentiments-0.1.0-py3-none-any.whl
quick-sentiments is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [13]:
from quick_sentiments import pre_process
from quick_sentiments import run_pipeline
from quick_sentiments import make_predictions

In [3]:
import polars as pl

In [4]:
df = pl.read_csv("quick_sentiments/demo/training_data/train.csv")

In [5]:
df.head(5)

movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
str,str,bool,str,str
"""marvelous_pirate""","""Benjamin Henry""",false,"""Henry Selick’s first movie sin…","""POSITIVE"""
"""tony_montana_frodo_baggins_v_r…","""Felicia Lopez""",false,"""With a cast that reads like th…","""NEGATIVE"""
"""darth_vader_katniss_everdeen_s…","""Mr. Charles Burgess""",true,"""Creed II does not give us anyt…","""POSITIVE"""
"""lara_croft_glimmer""","""Ryan Barrett""",false,"""I know what you're thinking, b…","""POSITIVE"""
"""jason_bourne_surreal_the_termi…","""Alexander Glover""",false,"""Director Fernando Meirelles te…","""POSITIVE"""


In [6]:
pre_process(df["reviewText"][0])

'henry selicks first movie since 2009s coraline fifth stopmotion masterpiece'

In [7]:
df = df.sample(fraction=0.1, with_replacement=False)
df = df.with_columns(
    pl.col("reviewText").map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
df.head(10)

C:\Users\meala\AppData\Local\Temp\ipykernel_14124\2017856843.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(


movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,processed
str,str,bool,str,str,str
"""majestic_the_chronicles_eclips…","""Ethan Dean""",true,"""This threatens to get precious…","""POSITIVE""","""threatens get precious time mo…"
"""mr._miyagi_holly_golightly""","""Wendy Johnson""",true,"""Children will love this. It's …","""POSITIVE""","""child love classic miyazaki"""
"""wonder_ellen_ripley_ellis_redd…","""Carlos Taylor""",false,"""There's never a dull moment in…","""POSITIVE""","""never dull moment gogo"""
"""ferris_bueller_evoke_han_solo""","""Amy Macdonald""",false,"""I enjoyed this film because I …","""POSITIVE""","""enjoyed film took grain salt v…"
"""tony_stark_surreal_scarlett_o'…","""Daniel Novak DDS""",false,"""Is deifying such a radical nat…","""POSITIVE""","""deifying radical nationalist w…"
"""katniss_everdeen_zephyr_termin…","""Alan Carson""",true,"""One thing is for sure: Directo…","""POSITIVE""","""one thing sure director mia ha…"
"""voyage_forrest_gump_emerald""","""Jose Stephens""",false,"""The film is a perfect example …","""NEGATIVE""","""film perfect example promising…"
"""fantastic_the_joker_v_katniss_…","""Nathan Simpson""",false,"""Every frame of this film feels…","""POSITIVE""","""every frame film feel meticulo…"
"""epic_hidden_marty_mcfly""","""Dana Cole""",false,"""Scott Cooper&#44; while still …","""NEGATIVE""","""scott cooper 44 still adept di…"


In [8]:
dt= run_pipeline(
    vectorizer_name="BOW", # BOW, tf, tfidf, wv
    model_name="logit", # logit, rf, XGB .#XGB takes long time, can not recommend using it on normal case
    df=df,
    text_column_name="reviewText",  # this is the column name of the text data, 
    sentiment_column_name = "sentiment",
    perform_tuning = True # make this true if you want to perform hyperparameter tuning, it will take longer time and 
                            # may run out of memory if the dataset is large,
)


--- Running Pipeline for Bow + Logit ---
Labels encoded: Original -> ['NEGATIVE' 'POSITIVE'], Encoded -> [0 1]
1. Vectorizing entire dataset (X)...
   - Generating Bag-of-Words features...
2. Splitting data into train/test...
3. Training and predicting...
   - Starting Logistic Regression training with GridSearchCV for hyperparameter tuning...
   - Using default parameter grid for tuning: {'solver': ['liblinear', 'lbfgs'], 'C': [0.1, 1.0, 10.0], 'class_weight': [None, 'balanced'], 'max_iter': [500, 1000]}
Fitting 5 folds for each of 24 candidates, totalling 120 fits

   - Best Hyperparameters found:
{'C': 1.0, 'class_weight': 'balanced', 'max_iter': 500, 'solver': 'liblinear'}
   - Best Cross-Validation Score (F1-weighted): 0.7558
Best model parameters: {'C': 1.0, 'class_weight': 'balanced', 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': 42, 'solver': 'libline

In [10]:
new_data = pl.read_csv("quick_sentiments/demo/new_data/test.csv",encoding='ISO-8859-1') #keep your file here
print(new_data.shape)
new_data= new_data.sample(fraction=0.25, shuffle=True, seed=42)
print(new_data.shape)

(55315, 4)
(13828, 4)


In [12]:
new_data = new_data.with_columns(
    pl.col("reviewText").map_elements(lambda x: pre_process(x, remove_brackets=True)).alias("processed")  #add inside the map_elements
)
new_data.head(10)

C:\Users\meala\AppData\Local\Temp\ipykernel_14124\3805368646.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  new_data = new_data.with_columns(


movieid,reviewerName,isTopCritic,reviewText,processed
str,str,bool,str,str
"""frodo_baggins_rocky_balboa_she…","""Toni Vaughn""",false,"""The result is an unsettling ta…","""result unsettling tale human p…"
"""stardust_john_mcclane""","""Carol Jennings""",false,"""Think twice about getting invo…","""think twice getting involved w…"
"""hermione_granger_sherlock_holm…","""Tara Huang""",true,"""A film that's so bloody wonder…","""film bloody wonderful meaning …"
"""astonish_valiant""","""Shelley Murillo""",false,"""...a decent setup that's emplo…","""decent setup employed progress…"
"""indiana_jones_dazzling_dorothy…","""Daniel Bond""",false,"""Inspiring? Not to me. Lamentab…","""inspiring lamentably bland exe…"
"""glimmer_hannibal_lecter_frodo_…","""Mrs. Nicole Fleming""",false,"""This perfectly executed piece …","""perfectly executed piece movie…"
"""brave_anakin_skywalker""","""Melissa Harrington""",false,"""The Richard Curtis script has …","""richard curtis script rooney m…"
"""katniss_everdeen_superman""","""Mckenzie Ortiz""",false,"""A fun-filled afternoon of dino…","""funfilled afternoon dinotastic…"
"""secret_magic_john_wick_legend""","""Samantha Ware""",false,"""I'm still not sure what this i…","""still sure supposed save bunch…"


In [14]:
make_predictions(
    new_data=new_data,
    text_column_name="processed",
    vectorizer=dt["vectorizer_object"],
    best_model=dt["model_object"],
    label_encoder=dt["label_encoder"],
    prediction_column_name="sentiment_predictions"  # Optional custom name
)

movieid,reviewerName,isTopCritic,reviewText,processed,sentiment_predictions
str,str,bool,str,str,str
"""frodo_baggins_rocky_balboa_she…","""Toni Vaughn""",false,"""The result is an unsettling ta…","""result unsettling tale human p…","""POSITIVE"""
"""stardust_john_mcclane""","""Carol Jennings""",false,"""Think twice about getting invo…","""think twice getting involved w…","""POSITIVE"""
"""hermione_granger_sherlock_holm…","""Tara Huang""",true,"""A film that's so bloody wonder…","""film bloody wonderful meaning …","""POSITIVE"""
"""astonish_valiant""","""Shelley Murillo""",false,"""...a decent setup that's emplo…","""decent setup employed progress…","""NEGATIVE"""
"""indiana_jones_dazzling_dorothy…","""Daniel Bond""",false,"""Inspiring? Not to me. Lamentab…","""inspiring lamentably bland exe…","""NEGATIVE"""
…,…,…,…,…,…
"""vivid_intrigue_celestial""","""Luke Reyes""",false,"""This is the film's ultimate me…","""film ultimate message nt think…","""POSITIVE"""
"""mystery_lost_james_t._kirk_sta…","""Kathy Wade""",false,"""It's the best Sondheim adaptat…","""best sondheim adaptation sayin…","""POSITIVE"""
"""hiccup_terminator_myriad_capta…","""Troy Watson""",false,"""The South Australian actor her…","""south australian actor deliver…","""POSITIVE"""
